In [1]:
# !export PYSPARK_SUBMIT_ARGS="--driver-memory 40G --executor-memory 8g pyspark-shell"
!echo $PYSPARK_SUBMIT_ARGS

--driver-memory 40G --executor-memory 8g pyspark-shell


In [2]:
import hail as hl
import pandas as pd

In [3]:
mt = hl.import_bgen('/vol/bmd/data/ukbiobank/genotypes/v3/ukb_imp_chr22_v3.bgen', entry_fields = ['GT'], sample_file = '/vol/bmd/data/ukbiobank/genotypes/v3/ukb19526_imp_chr1_v3_s487395.sample')

Initializing Spark and Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://nucleus.cels.anl.gov:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /vol/bmd/yanyul/GitHub/ptrs-ukb/notebook/hail-20191206-2351-0.2.28-61941242c15d.log
2019-12-06 23:51:41 Hail: INFO: Number of BGEN files parsed: 1
2019-12-06 23:51:41 Hail: INFO: Number of samples in BGEN files: 487409
2019-12-06 23:51:41 Hail: INFO: Number of variants across all BGEN files: 1255683


In [4]:
mt = mt.annotate_cols(eid=mt.s.replace("\_\d+", ""))
mt = mt.key_cols_by('eid')

In [5]:
mt.rows().select().show(5)

locus,alleles
locus<GRCh37>,array<str>
22:16050075,"[""A"",""G""]"
22:16050115,"[""G"",""A""]"
22:16050213,"[""C"",""T""]"
22:16050319,"[""C"",""T""]"
22:16050435,"[""T"",""C""]"


In [6]:
mt.row_key.show(5)

locus,alleles
locus<GRCh37>,array<str>
22:16050075,"[""A"",""G""]"
22:16050115,"[""G"",""A""]"
22:16050213,"[""C"",""T""]"
22:16050319,"[""C"",""T""]"
22:16050435,"[""T"",""C""]"


In [7]:
mt.s.show(5)

eid,s
str,str
"""2476612""","""2476612"""
"""5595764""","""5595764"""
"""5172041""","""5172041"""
"""3487211""","""3487211"""
"""2017223""","""2017223"""


In [8]:
f = open('/homes/yanyul/query_first_attempt_with_qc.csv', 'r')
header = f.readlines()[0].strip().split(',')
f.close()
list_of_str = ['eid', 'meaning', 'ethnicity_agg']
header_dic = {}
for i in header:
    if i in list_of_str:
        header_dic[i] = hl.tstr
    else:
        header_dic[i] = hl.tfloat64

In [9]:
pheno = (hl.import_table('/homes/yanyul/query_first_attempt_with_qc.csv', types = header_dic, delimiter = ',').key_by('eid'))

2019-12-06 23:51:56 Hail: INFO: Reading table with no type imputation
  Loading column 'eid' as type 'str' (user-specified)
  Loading column 'age_recruitment' as type 'float64' (user-specified)
  Loading column 'sex' as type 'float64' (user-specified)
  Loading column 'pc1' as type 'float64' (user-specified)
  Loading column 'pc2' as type 'float64' (user-specified)
  Loading column 'pc3' as type 'float64' (user-specified)
  Loading column 'pc4' as type 'float64' (user-specified)
  Loading column 'pc5' as type 'float64' (user-specified)
  Loading column 'pc6' as type 'float64' (user-specified)
  Loading column 'pc7' as type 'float64' (user-specified)
  Loading column 'pc8' as type 'float64' (user-specified)
  Loading column 'pc9' as type 'float64' (user-specified)
  Loading column 'pc10' as type 'float64' (user-specified)
  Loading column 'ethnicity_agg' as type 'str' (user-specified)
  Loading column 'meaning' as type 'str' (user-specified)
  Loading column 'height' as type 'float64' (

In [10]:
mt.count_rows()

1255683

In [11]:
pheno = pheno.filter(pheno.meaning == 'British')
pheno.count()

356476

In [12]:
mt = mt.annotate_cols(pheno = pheno[mt.eid])

In [13]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        eid: str, 
        pheno: struct {
            age_recruitment: float64, 
            sex: float64, 
            pc1: float64, 
            pc2: float64, 
            pc3: float64, 
            pc4: float64, 
            pc5: float64, 
            pc6: float64, 
            pc7: float64, 
            pc8: float64, 
            pc9: float64, 
            pc10: float64, 
            ethnicity_agg: str, 
            meaning: str, 
            height: float64, 
            dbp: float64, 
            sbp: float64, 
            bmi: float64, 
            wbc: float64, 
            rbc: float64, 
            hb: float64, 
            ht: float64, 
            mcv: float64, 
            mch: float64, 
            mchc: float64, 
            platelet: float64, 
            lymphocyte: float64, 
            monocyte: float64, 
            neutrophil: float64, 
            eosinophil: float64,

In [14]:
print(pheno.count())
print(mt.count_cols())

356476
487409


In [15]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.meaning))

{'British': 356476, None: 130933}

In [16]:
# snp_counts = mt.aggregate_rows(hl.agg.counter(hl.Struct(ref=mt.alleles[0], alt=mt.alleles[1])))

In [17]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        eid: str, 
        pheno: struct {
            age_recruitment: float64, 
            sex: float64, 
            pc1: float64, 
            pc2: float64, 
            pc3: float64, 
            pc4: float64, 
            pc5: float64, 
            pc6: float64, 
            pc7: float64, 
            pc8: float64, 
            pc9: float64, 
            pc10: float64, 
            ethnicity_agg: str, 
            meaning: str, 
            height: float64, 
            dbp: float64, 
            sbp: float64, 
            bmi: float64, 
            wbc: float64, 
            rbc: float64, 
            hb: float64, 
            ht: float64, 
            mcv: float64, 
            mch: float64, 
            mchc: float64, 
            platelet: float64, 
            lymphocyte: float64, 
            monocyte: float64, 
            neutrophil: float64, 
            eosinophil: float64,

In [18]:
mt = hl.sample_qc(mt)

In [19]:
mt.col.describe()

--------------------------------------------------------
Type:
        struct {
        s: str, 
        eid: str, 
        pheno: struct {
            age_recruitment: float64, 
            sex: float64, 
            pc1: float64, 
            pc2: float64, 
            pc3: float64, 
            pc4: float64, 
            pc5: float64, 
            pc6: float64, 
            pc7: float64, 
            pc8: float64, 
            pc9: float64, 
            pc10: float64, 
            ethnicity_agg: str, 
            meaning: str, 
            height: float64, 
            dbp: float64, 
            sbp: float64, 
            bmi: float64, 
            wbc: float64, 
            rbc: float64, 
            hb: float64, 
            ht: float64, 
            mcv: float64, 
            mch: float64, 
            mchc: float64, 
            platelet: float64, 
            lymphocyte: float64, 
            monocyte: float64, 
            neutrophil: float64, 
            eosinophil: float64,

In [20]:

# p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
# hail.plot.show(p)

In [21]:
mt = hl.variant_qc(mt)

In [22]:
mt.row.describe()

--------------------------------------------------------
Type:
        struct {
        locus: locus<GRCh37>, 
        alleles: array<str>, 
        rsid: str, 
        varid: str, 
        variant_qc: struct {
            AC: array<int32>, 
            AF: array<float64>, 
            AN: int32, 
            homozygote_count: array<int32>, 
            call_rate: float64, 
            n_called: int64, 
            n_not_called: int64, 
            n_filtered: int64, 
            n_het: int64, 
            n_non_ref: int64, 
            het_freq_hwe: float64, 
            p_value_hwe: float64
        }
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


In [ ]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)
print(mt.count_rows())

In [ ]:
# mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6)
print(mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6).count_rows())

In [ ]:
gwas = hl.linear_regression_rows(y = [ mt.pheno.bmi, mt.pheno.height ],
                                 x = mt.GT.n_alt_alleles(),
                                 covariates=[ 1.0, mt.pheno.age_recruitment, mt.pheno.sex, mt.pheno.sex * mt.pheno.age_recruitment ])
# gwas2 = hl.linear_regression_rows(y = mt.pheno.height,
#                                  x = mt.GT.n_alt_alleles(),
#                                  covariates=[ 1.0, mt.pheno.age_recruitment, mt.pheno.sex, mt.pheno.sex * mt.pheno.age_recruitment ])

In [ ]:
# tmp = gwas.export('./first_attempt_on_hail.output.tsv.bgz')

In [27]:
gwas.describe()
gwas.export('./first_attempt_on_hail.output.tsv.gz')

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh37> 
    'alleles': array<str> 
    'n': int32 
    'sum_x': float64 
    'y_transpose_x': array<float64> 
    'beta': array<float64> 
    'standard_error': array<float64> 
    't_stat': array<float64> 
    'p_value': array<float64> 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


2019-12-06 22:29:02 Hail: INFO: merging 293 files totalling 13.9M...
2019-12-06 22:29:02 Hail: INFO: while writing:
    ./first_attempt_on_hail.output.tsv.gz
  merge time: 263.452ms
